In [1]:
import openmeteo_requests
import numpy as np
import requests_cache
import pandas as pd
from retry_requests import retry


In [2]:
data_frame_storage_past_forecast=pd.DataFrame(columns=['date','temperature_2m', 'latitude', "longitude"])
		# Setup the Open-Meteo API client with cache and retry on error
for i in np.arange(43,50,2):
	for j in np.arange(-5,8,2):
				cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
				retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
				openmeteo = openmeteo_requests.Client(session = retry_session)

				# Make sure all required weather variables are listed here
				# The order of variables in hourly or daily is important to assign them correctly below
				url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
				params = {
					"latitude": i,
					"longitude": j,
					"start_date": "2023-01-01",
					"end_date": "2023-12-31",
					"hourly": "temperature_2m",
					"models": "meteofrance_seamless"
				}
				responses = openmeteo.weather_api(url, params=params)

				# Process first location. Add a for-loop for multiple locations or weather models
				response = responses[0]
				# Process hourly data. The order of variables needs to be the same as requested.
				hourly = response.Hourly()
				hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

				hourly_data = {"date": pd.date_range(
					start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
					end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
					freq = pd.Timedelta(seconds = hourly.Interval()),
					inclusive = "left"
				)}
				hourly_data["temperature_2m"] = hourly_temperature_2m


				hourly_dataframe = pd.DataFrame(data = hourly_data)
				hourly_dataframe.dropna(inplace=True)
				hourly_dataframe["latitude"] = [i for k in range(len(hourly_dataframe))]
				hourly_dataframe["longitude"] = [j for k in range(len(hourly_dataframe))]
				data_frame_storage_past_forecast = pd.concat([data_frame_storage_past_forecast, hourly_dataframe], axis=0)

nom='hourly_past_predicted_temperature_2m_2023.csv'
data_frame_storage_past_forecast.to_csv(nom)


C:\Users\tsang\AppData\Local\Temp\ipykernel_20416\1392581642.py:41: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_frame_storage_past_forecast = pd.concat([data_frame_storage_past_forecast, hourly_dataframe], axis=0)
